In [35]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition

df= pd.read_csv("EEG_data_confusion.csv")
mean_Att = round(df["Attention"].mean(), 3)
mean_Med = round(df["Mediation"].mean(), 3)

new = pd.DataFrame(columns = {"('Attention', " + str(mean_Att) + ')', "('Mediation', " + str(mean_Med) + ')'})
new.to_csv("output.csv", index = False)

df_scaled = StandardScaler().fit_transform(df)
pca = decomposition.PCA(n_components = 5)
Transformed_df = pca.fit_transform(df_scaled)
var = pca.explained_variance_ratio_
var = round(var.sum()*100, 2)
out = pd.DataFrame(columns = {"Variance Explained by 5 PCA Components is " + str(var) + '%'})
out.to_csv("output1.csv", index = False)

In [27]:
import pandas as pd
import numpy as np
from apyori import apriori

def uRecords():
    df = pd.read_csv("Symptoms.csv")
    df["Records"] = '[' + df['Symptoms'].astype(str) + ']'
    rec = pd.Series(df['Records'])
    return rec

def nUsers(Symptom):
    df = pd.read_csv("Symptoms.csv")
    count = df['Symptoms'].str.count(Symptom).sum()
    return count

def commonSymptom(Symptom):
    association_rules = apriori(transactions = uRecords(), min_support=0.0002, min_confidence=0.002, min_lift=1.1, min_length=2, max_length=2)
    association_results = list(association_rules)
    print(association_results)     


In [ ]:
import numpy as np
import pandas as pd
import nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

df = pd.read_csv("CricketMatchDataset.csv")
mean = df.agg({"Team 1 20ovr Score" : np.mean}).values
mean = round(*mean, 3)

count = df["Final Result"].str.contains(r'Home Team').sum()

df1 = df[['Home Team', 'Team 1 Batsman 1 Score']]
df2 = df1.groupby('Home Team', as_index = False).mean()
df_i = df2[df2['Home Team'] == 'India']
avg_i = round(df_i["Team 1 Batsman 1 Score"].iloc[0], 3)

df1 = df[((df['Home Team'] == 'India') & (df['Team Batting First'] == 'Home Team'))]
df2 = df[((df['Away Team'] == 'India') & (df['Team Batting First'] == 'Away Team'))]
df3 = df1.append(df2)
mean_i = round(df3["Team 1 Batsman 1 Score"].mean(axis = 0), 3)

df.drop(['Date of Match'], axis = 1, inplace = True)
X = df.drop('Final Result', axis = 1)
X = pd.get_dummies(X, prefix_sep = '_', drop_first = False)
no_of_col = len(X.columns)

labelencoder = LabelEncoder()
y = labelencoder.fit_transform(df['Final Result'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
lr_cric = LogisticRegression(C = 1.0, solver = 'liblinear', multi_class = 'ovr')
lr_cric.fit(X_train, y_train)
y_pred = lr_cric.predict(X_test)
acc = round(accuracy_score(y_test, y_pred), 3)

with open("output.csv", mode = "w") as f:
    f.write(str(mean))
    f.write("\n")
    f.write(str(count))
    f.write("\n")
    f.write(str(avg_i))
    f.write("\n")
    f.write(str(mean_i))
    f.write("\n")
    f.write(str(no_of_col))
    f.write("\n")
    f.write(str(acc))    

In [27]:
import re
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from nltk.tokenize import word_tokenize 
from nltk.stem.wordnet import WordNetLemmatizer

df = pd.read_csv("songs.csv")

#df['text'] = df['text'].apply(lambda x: re.sub('[!"#$%&\'()*+,-./:;<->?@[\\]^_`{|}~]'))
my_stop = set(stopwords.words('english') + [x for x in punctuation])
vectorizer = CountVectorizer(min_df = 20, max_df = 500, lowercase = True, stop_words = my_stop)

new_vector = vectorizer.fit_transform(df['text'])
rows, columns = new_vector.shape

clusters = KMeans(n_clusters = 4, random_state = 2)
clusters.fit(new_vector)

df['labels'] = clusters.labels_
dfk = df.groupby('labels').describe().transpose()
no_large = dfk.iloc[4][1]
no_NY = dfk.iloc[4][2]
no_artist = dfk.iloc[2][1]

with open("output.csv", mode = "w") as f:
    f.write(str(columns))
    f.write("\n")
    f.write("\n")
    f.write(str(no_large))
    f.write("\n")
    f.write(str(no_NY))
    f.write("\n")
    f.write(str(no_artist))